# Semantic Search with Cohere and Pinecone

In this notebook we will demonstrate how to perform semantic search for identifying similar or duplicate questions using Cohere and Pinecone.

![Steps in semantic search process](https://raw.githubusercontent.com/pinecone-io/examples/master/integrations/cohere/assets/index_query_pinecone_cohere.png)

## Setup

We first need to setup our environment and retrieve API keys for Cohere and Pinecone. Let's start with our environment, we need HuggingFace *Datasets* for our data, and the Cohere and Pinecone clients:

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy cohere pinecone-client


[notice] A new release of pip available: 22.2.2 -> 23.1.1
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip


And sign up for an API key over at [Cohere](https://os.cohere.ai/) and [Pinecone](https://app.pinecone.io), you can enter the keys directly in the cell below.

In [1]:
COHERE_KEY = 'nuyihjgwcybIE7SVhKtMw7zRp3vLuRJY94SsMKyl'
PINECONE_KEY = '2f14cfcd-33fa-453c-afc2-a9599563894a'

## Create Embeddings

We can create sentence embeddings easily using Cohere. First, we import the Cohere client and initialize our connection using the API key we retrieved earlier.

In [19]:
import cohere

co = cohere.Client(COHERE_KEY)

We will load the **T**ext **RE**trieval **C**onference (TREC) question classification dataset which contains 5.5K labeled questions. We will take the first 1K samples for this demo, but this can be scaled to millions or even billions of samples.

In [20]:
filename = "style.txt"
with open(filename) as file:
    lines = [line.strip() for line in file]
txt = " ".join(lines)

print(len(txt))

495978


In [21]:
print(lines[0])

Ten PrinciPles for


In [22]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(txt)
print(len(tokens))

96212


In [23]:
line_nums = [len(" ".join(line)) for line in lines]
def getLineNum(c_seen):
    for i in range(len(line_nums)):
        if c_seen > line_nums[i]:
            c_seen -= line_nums[i]
        else:
            return i

In [55]:
chunks = []
characters_seen = 0
chunk_size = 50
overlap = 5
for i in range(0, len(tokens), chunk_size-overlap):
    cur_chunk_tokens = tokens[i:i+chunk_size]
    fro = getLineNum(characters_seen)
    characters_seen += len("".join(cur_chunk_tokens))
    to = getLineNum(characters_seen)
    chunks.append({
        'text': " ".join(cur_chunk_tokens),
        'loc': {'from': fro, 'to': to}
        }
    )
    
print(len(chunks))
print(chunks[0])

2139
{'text': 'Ten PrinciPles for WriTing clearly 1 . Distinguish real grammatical rules from folklore ( pp . 10–21 ) . 2 . Use subjects to name the characters in your story ( pp . 46–52 ) . 3 . Use verbs to name their important actions ( pp . 32–39 )', 'loc': {'from': 0, 'to': 2}}


In [56]:
embeds = co.embed(
    texts=[chunk['text'] for chunk in chunks],
    model='small',
    truncate='LEFT'
).embeddings

In [57]:
shape = len(embeds)
print(shape)

2139


In [58]:
import pinecone
pinecone.init(
    PINECONE_KEY,
    environment="northamerica-northeast1-gcp"  # find next to API key in console
)
print('pinecone initialized')

pinecone initialized


In [59]:
index = pinecone.Index('cohere-index')
print('pinecone index initialized')

pinecone index initialized


In [60]:
batch_size = 128

ids = [str(i) for i in range(shape)]
# create list of metadata dictionaries
meta = [{'text': chunk['text'], 'source': "", "pdf_numpages": 695, 'line-from': chunk['loc']['from'], 'line-to': chunk['loc']['to']} for chunk in chunks]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

i_start = 0
for i in range(i_start, shape, batch_size):
    print(i)
    i_end = min(i+batch_size, shape)
    
#     print(to_upsert[i:i_end])
    
    index.upsert(vectors=to_upsert[i:i_end],
             namespace='english3')
#     time.sleep(1)

0
128
256
384
512
640
768
896
1024
1152
1280
1408
1536
1664
1792
1920
2048


We can then pass these questions to Cohere to create embeddings.

---